Following codes are based on following document, opened by the auther.

https://giggleliu.github.io/TwoQubit-VQE.html

This notebook is aimed to understand quantum tensor network and research the performance and building block technic.

# Solving TFI model with only 2 qubits - the 幺 simulation

Reference: Variational Quantum Eigensolver with Fewer Qubits

Jin-Guo Liu, Yi-Hong Zhang, Yuan Wan, Lei Wang

In [1]:
using Yao
using Statistics: mean
using LinearAlgebra
using Plots
using Random
rng = Random.GLOBAL_RNG

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


Random._GLOBAL_RNG()

### Build a quantum circuit inspired by MPS

The goal of this section is to build the MPS-inspired sampler as our ansatz

In [2]:
rotor(noleading::Bool=false, notrailing::Bool=false) = noleading ? (notrailing ? Rx(0) : chain(Rx(0), Rz(0))) : (notrailing ? chain(Rz(0), Rx(0)) : chain(Rz(0), Rx(0), Rz(0)))

# TT: Rx(0), TF: Rx(0)Rz(0), FT: Rz(0), FF: Rz(0)Rx(0)Rz(0)
# 0にはパラメータが後で入る

"""
    twoqubit_circuit(nlayer::Int, nrepeat::Int)

Construct the above ansatz, `nrepeat` is the number of measure operations,
`nlayer` is the length of each block.
"""
function twoqubit_circuit(nlayer::Int, nrepeat::Int, operator)
    nbit_measure = nbit_virtual = 1
    nbit_used = nbit_measure + nbit_virtual # = 2
    circuit = chain(nbit_used)

    for i=1:nrepeat
        unit = chain(nbit_used)
        #push!(unit, put(nbit_used, 2=>H))
        for j=1:nlayer
            push!(unit, put(nbit_used, 1=>rotor(true, false)))
            push!(unit, put(nbit_used, 2=>H))
            push!(unit, put(nbit_used, 2=>Rz(0.0)))
            push!(unit, control(nbit_used, 1, 2=>shift(0.0)))
            if j == nlayer
                push!(unit, put(nbit_used, 1=>rotor(true, false)))
                push!(unit, put(nbit_used, 2=>H))
                push!(unit, put(nbit_used, 2=>Rz(0.0)))
            end
        end
        push!(unit, Measure(nbit_used; operator=operator, locs=(1,), resetto=0))
        # Measure(n::Int; rng=Random.GLOBAL_RNG, operator=ComputationalBasis(), locs=AllLocs(), resetto=nothing, remove=false)
        # resetto: post measured state
        if i==nrepeat # last
            for k=2:nbit_used
                push!(unit, Measure(nbit_used; operator=operator, locs=(k,), resetto=0))
            end
        end
        push!(circuit, unit)
    end
    dispatch!(circuit, :random)
end

twoqubit_circuit

In [3]:
circuit = twoqubit_circuit(1, 3, X)

nqubits: 2
chain
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.8771358485495226)
│  │     └─ rot(Z, 0.2756739872069689)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.805493863162603)
│  ├─ control(1)
│  │  └─ (2,) shift(0.45436575653503897)
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.5240246989209438)
│  │     └─ rot(Z, 0.01119252117022107)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.7050678787619751)
│  └─ Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
├─ chain
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.6978939012726189)
│  │     └─ rot(Z, 0.5159484148134901)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.9331675731964206)
│  ├─ control(1)
│  │  └─ (2,) shift(0.9983012336416228)
│  ├─ put on (1)
│  │  └─ chain
│  │     ├─ rot(X, 0.20030369916973667)
│  │     └─ rot(Z, 0.7201665012983591)
│  ├─ put on (2)
│  │  └─ H
│  ├─ put on (2)
│  │  └─ rot(Z, 0.2516507507

In [5]:
length(parameters(circuit))

21

In [7]:
"""
    gensample(circuit, operator; nbatch=1024) -> Vector of Measure

Generate samples from MPS-inspired circuit. Here, `nbatch` means nshot.
`operator` is the operator to measure.
This function returns a vector of `Measure` gates, results are stored in `m.results`.
"""
function gensample(circuit; nbatch=1024)
    mblocks = collect_blocks(Measure, circuit) # collect all measurement block in circuit
    reg = zero_state(nqubits(circuit); nbatch=nbatch)
    reg |> circuit
    return mblocks
end

gensample

In [8]:
println(collect_blocks(Measure, circuit))
collect_blocks(Measure, circuit)[1].operator

Measure[Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎)), Measure(2;operator=X, locs=(2,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))]


X

In [9]:
res = gensample(circuit; nbatch=1024)

4-element Array{Measure,1}:
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(1,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))
 Measure(2;operator=X, locs=(2,), postprocess=ResetTo{BitBasis.BitStr{1,Int64}}(0 ₍₂₎))

In [10]:
res[4].results

1024-element Array{Complex{Float64},1}:
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
      ⋮
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
 -1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
  1.0 + 0.0im
 -1.0 + 0.0im
  1.0 + 0.0im
 -1.0 + 0.0im
  1.0 + 0.0im

# Model Hamiltonians

## Transverse field Ising Model

$H = \sum^{N-1}_{i=1} s^{z}_{i}s^{z}_{i+1} + h\sum^{N}_{i=1}s^{x}_{i}$

In [11]:
"""
for simplicity, we require an AbstractModel contains `size` and `periodic` members.
"""
abstract type AbstractModel{D} end

nspin(model::AbstractModel) = prod(model.size)

nspin (generic function with 1 method)

In [12]:
"""
transverse field ising model, `h` is the strength of transverse field.
"""
struct TFI{D} <:AbstractModel{1}
    size::NTuple{D, Int}
    h::Float64
    periodic::Bool
    TFI(size::Int...; h::Real, periodic::Bool) = new{length(size)}(size, Float64(h), periodic)
end

TFI

In [13]:
function get_bonds(model::AbstractModel{1}) # 1-D
    nbit, = model.size
    [(i, i%nbit+1) for i in 1:(model.periodic ? nbit : nbit-1)]
end

function get_bonds(model::AbstractModel{2}) # 2-D
    m, n = model.size
    cis = LinearIndices(model.size)
    bonds = Tuple{Int, Int, Float64}[]
    for i=1:m, j=1:n
        (i!=m || model.periodic) && push!(bonds, (cis[i,j], cis[i%m+1,j]))
        (j!=n || model.periodic) && push!(bonds, (cis[i,j], cis[i,j%n+1]))
    end
    bonds
end

get_bonds (generic function with 2 methods)

In [14]:
function hamiltonian(model::TFI{1})
    nbit = nspin(model)
    sum(repeat(nbit, Z, (i,j)) for (i,j) in get_bonds(model))*0.25 + # 1/4 * Z_i * Z_j
    sum(put(nbit, i=>X) for i=1:nbit)*0.5model.h # 1/2 * X_i
end

hamiltonian (generic function with 1 method)

In [15]:
tfi_model = TFI(4; h=0.5, periodic=false)

TFI{1}((4,), 0.5, false)

In [16]:
tfi_model.size

(4,)

In [17]:
tfi_h = hamiltonian(tfi_model)

nqubits: 4
+
├─ [scale: 0.25] +
│     ├─ +
│     │  ├─ repeat on (1, 2)
│     │  │  └─ Z
│     │  └─ repeat on (2, 3)
│     │     └─ Z
│     └─ repeat on (3, 4)
│        └─ Z
└─ [scale: 0.25] +
      ├─ +
      │  ├─ +
      │  │  ├─ put on (1)
      │  │  │  └─ X
      │  │  └─ put on (2)
      │  │     └─ X
      │  └─ put on (3)
      │     └─ X
      └─ put on (4)
         └─ X


In [18]:
function ising_energy(circuit, nlayer::Int, nrepeat::Int, bonds, basis; nbatch=nbatch)
    circuit_with_basis = circuit(nlayer, nrepeat, basis)
    mblocks = gensample(circuit_with_basis; nbatch=nbatch) # get measurement blocks and do sampling
    nspin = length(mblocks)
    local eng = 0.0
    for (a, b) in bonds
        eng += mean(mblocks[a].results .* mblocks[b].results)
    end
    eng/=4
    return eng, parameters(circuit_with_basis)
end

function energy(circuit, nlayer::Int, nrepeat::Int, model::TFI; nbatch=1024)
    # measuring Z
    eng, params = ising_energy(circuit, nlayer, nrepeat, get_bonds(model), Z; nbatch=nbatch)
    # measuring X
    circuit_with_basis = twoqubit_circuit(nlayer, nrepeat, X)
    dispatch!(circuit_with_basis, params)
    mblocks = gensample(circuit_with_basis; nbatch=nbatch)
    engx = sum(mean.([m.results for m in mblocks]))
    eng + model.h*engx/2
end

energy (generic function with 1 method)

In [19]:
energy(twoqubit_circuit, 1, 3, tfi_model; nbatch=100000)

0.83691 + 0.0im

## Heisenberg Model

$H = \sum^{N-1}_{\alpha = x,y,z;i=1} s^{\alpha}_{i}s^{\alpha}_{i+1}$

In [20]:
struct Heisenberg{D} <: AbstractModel{D}
    size::NTuple{D, Int}
    periodic::Bool
    Heisenberg(size::Int...; periodic::Bool) = new{length(size)}(size, periodic)
end

In [21]:
heisenberg_ij(nbit::Int, i::Int, j::Int=i+1) = put(nbit, i=>X)*put(nbit, j=>X) + put(nbit, i=>Y)*put(nbit, j=>Y) + put(nbit, i=>Z)*put(nbit, j=>Z)
function hamiltonian(model::Heisenberg)
    nbit = nspin(model)
    sum(x->heisenberg_ij(nbit, x[1], x[2]), get_bonds(model))*0.25
end

hamiltonian (generic function with 2 methods)

In [22]:
function ising_energy(circuit, nlayer::Int, nrepeat::Int, bonds, basis; params=nothing, nbatch=nbatch)
    circuit_with_basis = circuit(nlayer, nrepeat, basis)
    if params != nothing
        dispatch!(circuit_with_basis, params)
    end
    mblocks = gensample(circuit_with_basis; nbatch=nbatch) # get measurement blocks and do sampling
    nspin = length(mblocks)
    local eng = 0.0
    for (a, b) in bonds
        eng += mean(mblocks[a].results .* mblocks[b].results)
    end
    eng/=4
    return eng, parameters(circuit_with_basis)
end

function energy(circuit, nlayer::Int, nrepeat::Int, model::Heisenberg; nbatch=1024)
    bonds = get_bonds(model)
    isingX, params = ising_energy(circuit, nlayer, nrepeat, bonds, X; nbatch=nbatch)
    isingY, params = ising_energy(circuit, nlayer, nrepeat, bonds, X; params=params, nbatch=nbatch)
    isingZ, params = ising_energy(circuit, nlayer, nrepeat, bonds, X; params=params, nbatch=nbatch)
    eng = isingX + isingY + isingZ
end

energy (generic function with 2 methods)

In [23]:
hei_model = Heisenberg(4; periodic=false)

Heisenberg{1}((4,), false)

In [24]:
get_bonds(hei_model)

3-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 4)

In [113]:
energy(twoqubit_circuit, 1, 3, hei_model)

0.77392578125 + 0.0im

# Build the expanded view and check the energy

The expanded view is the equivalent model without qubit reusing, it is used for testing purpose. From this circuit, we are able to obtain the ground state wave function directly.

In [121]:
function expand_circuit(circuit)
    nbit = length(collect_blocks(Measure, circuit))
    nm = 1
    nv = 1
    c = chain(nbit)
    for (i, blk) in enumerate(circuit) # circuit consists of 3 unit (chain)
        blk = chain([b for b in blk if !(b isa Measure)]...)
        push!(c, subroutine(nbit, blk, [(i-1)*nm+1:i*nm..., nbit-nv+1:nbit...])) # subroutine(n, block, locs)
    end
    c
end

expand_circuit (generic function with 1 method)

In [217]:
circuit = twoqubit_circuit(1, 4, Z)
hei_model = Heisenberg(5; periodic=false)

nm=1
nv=1
nbit=4
i=3
blk = chain([b for b in circuit[i] if !(b isa Measure)]...)
subroutine(nbit, blk, [(i-1)*nm+1:i*nm..., nbit-nv+1:nbit...])
# locs=[1, 4]なら、元々qubit1, 2にかかってたゲートがqubit1, 4にかかる

nqubits: 4
Subroutine: (3, 4)
└─ chain
   ├─ put on (1)
   │  └─ chain
   │     ├─ rot(X, 0.5446317385074191)
   │     └─ rot(Z, 0.3472208675835622)
   ├─ put on (2)
   │  └─ H
   ├─ put on (2)
   │  └─ rot(Z, 0.5889628499535176)
   ├─ control(1)
   │  └─ (2,) shift(0.023556964519714008)
   ├─ put on (1)
   │  └─ chain
   │     ├─ rot(X, 0.39404519902365376)
   │     └─ rot(Z, 0.5945383227685819)
   ├─ put on (2)
   │  └─ H
   └─ put on (2)
      └─ rot(Z, 0.8629551112988589)


In [218]:
c_expand = expand_circuit(circuit)
# Measureを除いた回路

nqubits: 5
chain
├─ Subroutine: (1, 5)
│  └─ chain
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X, 0.13716780319521793)
│     │     └─ rot(Z, 0.28657772893632494)
│     ├─ put on (2)
│     │  └─ H
│     ├─ put on (2)
│     │  └─ rot(Z, 0.2873634372307643)
│     ├─ control(1)
│     │  └─ (2,) shift(0.2515348872209271)
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X, 0.9970313059194842)
│     │     └─ rot(Z, 0.7513890532354695)
│     ├─ put on (2)
│     │  └─ H
│     └─ put on (2)
│        └─ rot(Z, 0.8474654591206003)
├─ Subroutine: (2, 5)
│  └─ chain
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X, 0.5997859504360255)
│     │     └─ rot(Z, 0.513191347602107)
│     ├─ put on (2)
│     │  └─ H
│     ├─ put on (2)
│     │  └─ rot(Z, 0.7512199824176007)
│     ├─ control(1)
│     │  └─ (2,) shift(0.9479019486778417)
│     ├─ put on (1)
│     │  └─ chain
│     │     ├─ rot(X, 0.6380836979697222)
│     │     └─ rot(Z, 0.2339451662517471)
│     ├─ put on (2)
│

In [219]:
function wave_function(circuit)
    #ec = chem2circuit(circuit)
    zero_state(nqubits(circuit)) |> circuit
end

wave_function (generic function with 1 method)

In [221]:
#@show expect(tfi_h, wave_function(c_expand)) |> real
@show expect(hamiltonian(hei_model), wave_function(c_expand)) |> real;

expect(hamiltonian(hei_model), wave_function(c_expand)) |> real = 0.8499529029520934


# Obtaining the ground state

Sequential optimization

In [222]:
function fidelity(circuit, VG)
    psi = zero_state(nqubits(circuit)) |> circuit
    abs(statevec(psi)' * VG)
end

fidelity (generic function with 1 method)

In [223]:
nparameters(circuit)

28

In [224]:
function loss(hamiltonian, circ)
    expect(hamiltonian, wave_function(circ)) |> real
    return real
end

loss (generic function with 1 method)

In [227]:
a, grad = expect'(hamiltonian(hei_model), zero_state(5) => c_expand)

ArrayReg{1, Complex{Float64}, Array...}
    active qubits: 5/5 => [-0.01877000297592657, -0.002512856895093666, -0.1638917637906169, 0.0012398070841330897, -0.014405577791108277, -0.019278376419091386, 0.03475485824801158, -0.10341649263840422, 0.024652483939704532, -0.19788785157403962  …  0.07215815232281776, 0.02374535174489048, -0.10860147001815784, 0.002607013194380975, 0.06513748264379735, 0.06636779254164607, 0.00691078526935412, -0.09087674350221894, 0.14078263290811094, -0.18242922467490802]

In [231]:
dispatch!(c_expand, :random)
for i in 1:10000
      _, grad = expect'(hamiltonian(hei_model), zero_state(5) => c_expand)
      dispatch!(-, c_expand, 0.05 * grad)
      #println("Step $i, energy = $(real.(expect(hamiltonian(hei_model), zero_state(4)=>c_expand)))")
end

println("energy = $(real.(expect(hamiltonian(hei_model), zero_state(5)=>c_expand)))")

using LinearAlgebra
w, _ = eigen(Matrix(mat(hamiltonian(hei_model))))
print(w)

energy = -1.3419433400299448
[-1.9278862533179943, -1.9278862533179937, -1.207106781186551, -1.207106781186545, -0.8090169943749483, -0.8090169943749471, -0.8090169943749452, -0.8090169943749452, -0.660818587131649, -0.6608185871316482, -0.3090169943749489, -0.3090169943749472, -0.3090169943749466, -0.3090169943749461, 0.20710678118654754, 0.20710678118654824, 0.3090169943749474, 0.30901699437494784, 0.309016994374948, 0.3090169943749484, 0.5887048404496422, 0.5887048404496428, 0.8090169943749469, 0.8090169943749471, 0.8090169943749473, 0.8090169943749477, 0.9999999999999998, 1.0, 1.0, 1.0, 1.0, 1.0000000000000002]

Ground stateまでいかない...パラメータ数が足りない？->ansatzの表現力

In [13]:
hei_model_2q = Heisenberg(2; periodic=false)
#hamiltonian(hei_model_2q)
using LinearAlgebra
w, s = eigen(Matrix(mat(hamiltonian(hei_model_2q))))
println(w)
println(s)

UndefVarError: UndefVarError: Heisenberg not defined

In [14]:
hamiltonian(hei_model_2q)

UndefVarError: UndefVarError: hamiltonian not defined

## Classical method for calculating and checking minimum eigen value of 2-qubit Heisenberg model

In [24]:
matH = [1 0 0 0; 0 -1 2 0; 0 2 -1 0; 0 0 0 1] * 0.25
w, vecs = eigen(Matrix(matH))

println(w)
println(vecs)

[-0.7499999999999994, 0.25, 0.25, 0.25]
[0.0 1.0 0.0 0.0; 0.7071067811865477 0.0 0.7071067811865475 0.0; -0.7071067811865475 0.0 0.7071067811865477 0.0; 0.0 0.0 0.0 1.0]


In [25]:
function tensordot(x::Array, y::Array)
    lx = size(x)
    ly = size(y)
    res = zeros(lx[1] * ly[1], lx[2] * ly[2])
    for i in 1:lx[1]
        for j in 1:lx[2]
            res[ (1+(i-1)*ly[1]):(ly[1]+(i-1)*ly[1]), (1+(j-1)*ly[2]):(ly[2]+(j-1)*ly[2]) ] = x[i,j] * y
        end
    end
    return res
end

tensordot (generic function with 1 method)

### checking for minimum eigenvector and value

In [26]:
using LinearAlgebra
state = vecs[:,1] # minimum eigen vector
px = [0 1;1 0]
py = [0 -1im;1im 0]
pz = [1 0;0 -1]
pxx = tensordot(px, px)
pyy = tensordot(py, py)
pzz = tensordot(pz, pz)

rho = state * transpose(state)
Exx = tr(rho * pxx)
Eyy = tr(rho * pyy)
Ezz = tr(rho * pzz)
(Exx + Eyy + Ezz)/4 # minimum eigen value

-0.7500000000000001

In [11]:

tr(px)

0